In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam

from torch.utils.data import TensorDataset,DataLoader

In [ ]:
class LSTM_S(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,n_layers,bias):
        super(LSTM_S,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.n_layers=n_layers
        self.bias=bias

        self.layers=nn.ModuleList()
        for i in range(n_layers):
            layer_input_size = input_size if i == 0 else hidden_size
            layer = nn.Module()
            layer.wlr1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wlr2 = nn.Parameter(torch.rand(layer_input_size), requires_grad=True)
            layer.blr1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wpr1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wpr2 = nn.Parameter(torch.rand(layer_input_size), requires_grad=True)
            layer.bpr1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wp1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wp2 = nn.Parameter(torch.rand(layer_input_size), requires_grad=True)
            layer.bp1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wo1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            layer.wo2 = nn.Parameter(torch.rand(layer_input_size), requires_grad=True)
            layer.bo1 = nn.Parameter(torch.rand(hidden_size), requires_grad=True)
            self.layers.append(layer)

    def unit_cal(self, layer, input, long_mm, short_mm):
        First_stage = torch.sigmoid(torch.mm(short_mm, layer.wlr1.t()) + torch.mm(input, layer.wlr2.t()) + layer.blr1)
        Second_stage = torch.sigmoid(torch.mm(short_mm, layer.wpr1.t()) + torch.mm(input, layer.wpr2.t()) + layer.bpr1)
        Second_memory = torch.tanh(torch.mm(short_mm, layer.wp1.t()) + torch.mm(input, layer.wp2.t()) + layer.bp1)
        updated_long_memory = ((long_mm * First_stage) + (Second_stage * Second_memory))
        output_percent = torch.sigmoid(torch.mm(short_mm, layer.wo1.t()) + torch.mm(input, layer.wo2.t()) + layer.bo1)         
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        return updated_long_memory, updated_short_memory
    
    def forward(self, input, hx=None):
        batch_size, seq_len, _ = input.size()
        
        if hx is None:
            h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
            c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
            if input.is_cuda:
                h0 = h0.cuda()
                c0 = c0.cuda()
        else:
            h0, c0 = hx
        
        outs = []
        hidden = [(h0[i], c0[i]) for i in range(self.num_layers)]
        
        for t in range(seq_len):
            for layer in range(self.num_layers):
                if layer == 0:
                    layer_input = input[:, t, :]
                else:
                    layer_input = hidden[layer-1][1]  
                
                long_memory, short_memory = self.unit_cal(
                    self.layers[layer],
                    layer_input,
                    hidden[layer][0], 
                    hidden[layer][1]   
                )
                hidden[layer] = (long_memory, short_memory)
            
            outs.append(hidden[-1][1]) 
        
        out = outs[-1].squeeze()
        out = self.fc(out)
        return out
        
        